# Pokémon Agent Tool Debugging Notebook

This notebook helps debug the tool calls used in the Pokémon Agent system. It allows testing each tool individually to verify functionality.

## Setup

First, we need to import the necessary modules.

In [ ]:
import os
import sys
import json
import time
import threading
from IPython.display import Image, display, clear_output
import ipywidgets as widgets
import anthropic

# Import our modules
from pokemon_emulator import GameEmulator
from pokemon_knowledge import KnowledgeBase
from pokemon_tools import PokemonTools

### Configure the ROM path and API key

In [ ]:
# Path to your Pokémon ROM file
ROM_PATH = "roms/Pokemon Red Version (Colorization)/Pokemon Red Version (Colorization).gb"  # Update this

# Anthropic API key (optional, only needed if testing with Claude)
API_KEY = os.environ.get("ANTHROPIC_API_KEY", "")  # Will use env var or empty string

## Interactive Game Controls

This section provides an interactive interface to control the game manually or through API calls.

In [ ]:
class InteractiveGameController:
    """Controller class for interactive game debugging"""
    
    def __init__(self, rom_path, headless=False, speed=1, sound=False):
        """Initialize the controller"""
        self.rom_path = rom_path
        self.headless = headless
        self.speed = speed
        self.sound = sound
        
        # Initialize components
        self.emulator = None
        self.kb = KnowledgeBase(save_path="debug_knowledge.json")
        self.tools = None
        
        # Control flags
        self.running = False
        self.game_thread = None
        self.auto_refresh = False
        self.refresh_thread = None
        self.paused = False
        
        # UI components
        self.status_label = None
        self.screen_output = None
        self.control_buttons = None
        self.game_state_output = None
        
    def start_game(self):
        """Start the game emulator"""
        if self.emulator is not None:
            print("Game is already running!")
            return
            
        print(f"Initializing game with ROM: {self.rom_path}")
        self.emulator = GameEmulator(self.rom_path, headless=self.headless, speed=self.speed, sound=self.sound)
        self.tools = PokemonTools(self.emulator, self.kb)
        
        # Start the game
        print("Starting game...")
        self.emulator.start_game(skip_intro=True)
        print("Game started successfully!")
        
        # Start game loop in a separate thread
        self.running = True
        self.game_thread = threading.Thread(target=self._game_loop)
        self.game_thread.daemon = True
        self.game_thread.start()
        
        # Update status
        if self.status_label:
            self.status_label.value = "<font color='green'><b>Game Running</b></font>"
            
        # Display initial screen
        self.update_screen()
        
    def stop_game(self):
        """Stop the game emulator"""
        if self.emulator is None:
            print("Game is not running!")
            return
            
        # Stop the game loop
        self.running = False
        if self.game_thread:
            self.game_thread.join(timeout=1.0)
            
        # Stop auto-refresh if running
        self.auto_refresh = False
        if self.refresh_thread:
            self.refresh_thread.join(timeout=1.0)
            
        # Close the emulator
        print("Stopping game...")
        self.emulator.close()
        self.emulator = None
        self.tools = None
        print("Game stopped.")
        
        # Update status
        if self.status_label:
            self.status_label.value = "<font color='red'><b>Game Stopped</b></font>"
    
    def _game_loop(self):
        """Background game loop that keeps the game running"""
        try:
            while self.running:
                if not self.paused:
                    # Just tick the game to keep it running
                    self.emulator.pyboy.tick()
                time.sleep(0.001)  # Small sleep to prevent CPU hogging
        except Exception as e:
            print(f"Error in game loop: {e}")
            self.running = False
    
    def toggle_pause(self):
        """Toggle game pause state"""
        if self.emulator is None:
            print("Game is not running!")
            return
            
        self.paused = not self.paused
        print(f"Game {'paused' if self.paused else 'resumed'}.")
    
    def press_button(self, button, hold_frames=10):
        """Press a button on the game"""
        if self.emulator is None:
            print("Game is not running!")
            return "Game not running"
            
        result = self.emulator.press_button(button, hold_frames)
        self.update_screen()
        self.update_game_state()
        return result
    
    def execute_tool(self, tool_name, params):
        """Execute a tool using the tools manager"""
        if self.emulator is None or self.tools is None:
            print("Game is not running!")
            return "Game not running"
            
        result = self.tools.execute_tool(tool_name, params)
        self.update_screen()
        self.update_game_state()
        return result
    
    def update_screen(self):
        """Update the displayed game screen"""
        if self.emulator is None:
            return
            
        # Get the screen as a PIL image
        screen = self.emulator.get_screen_pil()
        
        # Display it
        if self.screen_output:
            with self.screen_output:
                clear_output(wait=True)
                display(screen)
        else:
            display(screen)
    
    def update_game_state(self):
        """Update the displayed game state"""
        if self.emulator is None:
            return
            
        # Get the game state
        state = self.emulator.get_game_state()
        formatted_state = self.emulator.format_game_state(state)
        
        # Display it
        if self.game_state_output:
            self.game_state_output.value = f"<pre>{formatted_state}</pre>"
        else:
            print(formatted_state)
    
    def toggle_auto_refresh(self):
        """Toggle automatic screen refresh"""
        if self.emulator is None:
            print("Game is not running!")
            return
            
        self.auto_refresh = not self.auto_refresh
        
        if self.auto_refresh:
            print("Auto-refresh enabled.")
            # Start refresh thread
            self.refresh_thread = threading.Thread(target=self._refresh_loop)
            self.refresh_thread.daemon = True
            self.refresh_thread.start()
        else:
            print("Auto-refresh disabled.")
    
    def _refresh_loop(self):
        """Background loop for automatic screen refresh"""
        try:
            while self.auto_refresh and self.running:
                self.update_screen()
                self.update_game_state()
                time.sleep(0.5)  # Refresh every half second
        except Exception as e:
            print(f"Error in refresh loop: {e}")
            self.auto_refresh = False
    
    def save_state(self, filename="manual_save.state"):
        """Save the current game state"""
        if self.emulator is None:
            print("Game is not running!")
            return "Game not running"
            
        path = self.emulator.save_state(filename)
        print(f"Saved game state to {path}")
        return path
    
    def load_state(self, filename="manual_save.state"):
        """Load a saved game state"""
        if self.emulator is None:
            print("Game is not running!")
            return "Game not running"
            
        result = self.emulator.load_state(filename)
        self.update_screen()
        self.update_game_state()
        print(result)
        return result
    
    def create_ui(self):
        """Create an interactive UI for controlling the game"""
        # Status display
        self.status_label = widgets.HTML(
            value="<font color='red'><b>Game Not Running</b></font>",
            description="Status:"
        )
        
        # Game control buttons
        start_button = widgets.Button(description="Start Game", button_style="success")
        stop_button = widgets.Button(description="Stop Game", button_style="danger")
        pause_button = widgets.Button(description="Pause/Resume", button_style="warning")
        refresh_button = widgets.Button(description="Refresh Screen", button_style="info")
        auto_refresh_button = widgets.Button(description="Toggle Auto-Refresh", button_style="info")
        save_button = widgets.Button(description="Save State", button_style="info")
        load_button = widgets.Button(description="Load State", button_style="info")
        
        # Button actions
        start_button.on_click(lambda b: self.start_game())
        stop_button.on_click(lambda b: self.stop_game())
        pause_button.on_click(lambda b: self.toggle_pause())
        refresh_button.on_click(lambda b: (self.update_screen(), self.update_game_state()))
        auto_refresh_button.on_click(lambda b: self.toggle_auto_refresh())
        save_button.on_click(lambda b: self.save_state())
        load_button.on_click(lambda b: self.load_state())
        
        # Game control buttons
        control_row1 = widgets.HBox([start_button, stop_button, pause_button])
        control_row2 = widgets.HBox([refresh_button, auto_refresh_button, save_button, load_button])
        self.control_buttons = widgets.VBox([control_row1, control_row2])
        
        # Game pad buttons
        up_button = widgets.Button(description="↑", layout=widgets.Layout(width='50px', height='50px'))
        down_button = widgets.Button(description="↓", layout=widgets.Layout(width='50px', height='50px'))
        left_button = widgets.Button(description="←", layout=widgets.Layout(width='50px', height='50px'))
        right_button = widgets.Button(description="→", layout=widgets.Layout(width='50px', height='50px'))
        a_button = widgets.Button(description="A", layout=widgets.Layout(width='50px', height='50px'))
        b_button = widgets.Button(description="B", layout=widgets.Layout(width='50px', height='50px'))
        start_gb_button = widgets.Button(description="START", layout=widgets.Layout(width='80px', height='40px'))
        select_button = widgets.Button(description="SELECT", layout=widgets.Layout(width='80px', height='40px'))
        
        # Game pad button actions
        up_button.on_click(lambda b: self.press_button('up'))
        down_button.on_click(lambda b: self.press_button('down'))
        left_button.on_click(lambda b: self.press_button('left'))
        right_button.on_click(lambda b: self.press_button('right'))
        a_button.on_click(lambda b: self.press_button('a'))
        b_button.on_click(lambda b: self.press_button('b'))
        start_gb_button.on_click(lambda b: self.press_button('start'))
        select_button.on_click(lambda b: self.press_button('select'))
        
        # Arrange game pad
        empty = widgets.Button(description="", layout=widgets.Layout(width='50px', height='50px'), disabled=True)
        empty.style.button_color = 'white'
        
        dpad_row1 = widgets.HBox([empty, up_button, empty])
        dpad_row2 = widgets.HBox([left_button, empty, right_button])
        dpad_row3 = widgets.HBox([empty, down_button, empty])
        dpad = widgets.VBox([dpad_row1, dpad_row2, dpad_row3])
        
        buttons_row = widgets.HBox([b_button, a_button])
        menu_row = widgets.HBox([select_button, start_gb_button])
        action_buttons = widgets.VBox([buttons_row, menu_row])
        
        gamepad = widgets.HBox([dpad, widgets.Box(layout=widgets.Layout(width='50px')), action_buttons])
        
        # Screen and state display
        self.screen_output = widgets.Output(layout=widgets.Layout(width='320px', height='288px', border='1px solid gray'))
        self.game_state_output = widgets.HTML("<pre>Game not started</pre>")
        
        # Layout the UI
        status_row = widgets.HBox([self.status_label])
        control_panel = widgets.VBox([status_row, self.control_buttons])
        game_display = widgets.HBox([self.screen_output, self.game_state_output])
        
        # Final layout
        ui = widgets.VBox([control_panel, game_display, gamepad])
        display(ui)
        
        return ui

## Initialize the Interactive Game Controller

Run this cell to create and display the interactive game controller.

In [ ]:
controller = InteractiveGameController(ROM_PATH, headless=False, speed=1, sound=False)
controller.start_game()

In [ ]:
# Create the controller
controller = InteractiveGameController(ROM_PATH, headless=False, speed=1, sound=False)

# Create the UI
ui = controller.create_ui()

## Test Individual Tools

Now you can test individual tools while the game is running. Use these cells to test specific tool calls.

In [5]:
# Test press_button tool
result = controller.execute_tool("press_button", {"button": "a", "hold_frames": 10})
print(f"Result: {result}")

Game is not running!
Result: Game not running


In [ ]:
# Test wait_frames tool
result = controller.execute_tool("wait_frames", {"num_frames": 60})
print(f"Result: {result}")

In [ ]:
# Test update_knowledge tool
result = controller.execute_tool("update_knowledge", {
    "section": "game_state", 
    "key": "current_location", 
    "value": "Pallet Town"
})
print(f"Result: {result}")

# Verify the knowledge was stored
value = controller.tools.kb.get("game_state", "current_location")
print(f"Stored value: {value}")

In [ ]:
# Test get_game_info tool
result = controller.execute_tool("get_game_info", {"info_type": "player_position"})
print(f"Result: {result}")

## Test Action Sequence

Test a sequence of actions to simulate what Claude might do.

In [ ]:
def run_action_sequence(actions):
    """Run a sequence of actions and show results."""
    print("Running action sequence...")
    
    for i, (tool_name, params) in enumerate(actions, 1):
        print(f"\nStep {i}: Executing {tool_name} with params: {params}")
        result = controller.execute_tool(tool_name, params)
        print(f"Result: {result}")
        time.sleep(0.5)  # Short pause between actions
    
    print("\nSequence completed.")

In [ ]:
# Example sequence to navigate and interact
actions = [
    ("press_button", {"button": "a", "hold_frames": 10}),  # Talk/Interact
    ("wait_frames", {"num_frames": 30}),                     # Wait for text
    ("press_button", {"button": "a", "hold_frames": 10}),  # Advance text
    ("press_button", {"button": "down", "hold_frames": 15}), # Move down
    ("update_knowledge", {"section": "player_progress", "key": "talked_to_npc", "value": "true"})
]

run_action_sequence(actions)

## Test with Claude API (Optional)

This section tests sending a request to Claude with the current game state and executing the returned action.

In [ ]:
def call_claude_and_execute_action():
    """Call Claude API, get an action, and execute it"""
    if not API_KEY:
        print("No API key provided. Skipping Claude API call.")
        return None
        
    if controller.emulator is None:
        print("Game is not running!")
        return None
    
    client = anthropic.Anthropic(api_key=API_KEY)
    
    # Get game state and screen
    game_state = controller.emulator.get_game_state()
    state_text = controller.emulator.format_game_state(game_state)
    screen_base64 = controller.emulator.get_screen_base64()
    recent_actions = controller.tools.kb.get_recent_actions(5)
    
    # System prompt
    system_prompt = """
You are an expert Pokémon player. Analyze the game state and use a tool to take the best action.
First think step-by-step about the current situation, then use a tool to execute the appropriate action.
Always use a tool - do not just describe what to do.
"""
    
    # Message content
    message_content = [
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": screen_base64
            }
        },
        {
            "type": "text",
            "text": f"""Current game state:\n{state_text}\n\nRecent actions:\n{recent_actions}\n\nAnalyze the current situation and decide what action to take. Use one of the available tools."""
        }
    ]
    
    print("Calling Claude API...")
    
    # Make API call
    try:
        response = client.messages.create(
            model="claude-3-opus-20240229", 
            system=system_prompt,
            messages=[{"role": "user", "content": message_content}],
            max_tokens=1000,
            temperature=0.7,
            tools=controller.tools.define_tools()
        )
        
        print("\nClaude's response:")
        tool_call = None
        
        # Process the response
        for content in response.content:
            if content.type == "text":
                print(content.text)
            elif content.type == "tool_use":
                tool_call = content
                print(f"\n[Using tool: {content.name}]")
                print(f"Parameters: {content.input}")
        
        # Execute the tool if found
        if tool_call:
            print("\nExecuting Claude's suggested action...")
            result = controller.execute_tool(tool_call.name, tool_call.input)
            print(f"Result: {result}")
            controller.tools.kb.add_action(tool_call.name, result)
            return result
        else:
            print("\nNo tool call found in Claude's response.")
            return None
            
    except Exception as e:
        print(f"Error calling Claude API: {e}")
        return None

In [ ]:
# Test Claude API integration
call_claude_and_execute_action()

## Cleanup

Make sure to stop the game when you're done.

In [ ]:
# Clean up resources
controller.stop_game()
print("Controller shutdown complete.")